# Notes on SGP4 Libraries and Comparisons of their Propagators

# Some History
SGP4 is the propagator used strictly with the publicly available TLEs. The original code by the author is provided primarily at the website [here](https://celestrak.org/software/vallado-sw.php), but I have collated a copy in my repository [here](https://github.com/icyveins7/sgp4-download-celestrak) along with some notes about the two different versions available on the website. To make things clear, I will reference the versions of the code I reupload onto my repository: current test is at commit 09142b1.

This current repository, ```pydsproutines```, also contains some code from the python libraries ```sgp4``` (which is a simple wrapper around the SGP4 library) and ```skyfield``` (which has more functions, and itself wraps the python ```sgp4``` library).

# Comparisons
We would like to compare the results of the same propagation in the above libraries, bearing in mind that the calling structure is different; if the inputs required are different, we would like to determine what is required to make the results the same, if any.

# Coordinate Systems
Before we start, it is important to understand the different coordinate systems at play here. The TLEs and the SGP4 propagator are closely intertwined, and both are expected to operate _solely_ in what is known as the __TEME (True Equator Mean Equinox)__ coordinate system.

However, for most intents and purposes, we require the __ITRF (International Terrestrial Reference Frame)__, which is a specification of an __ECEF (Earth-Centered Earth-Fixed)__ frame. This conversion from TEME to ITRF is _non-trivial_, and is the part which requires a lot of external input i.e. not just the TLE, but other things known as Earth Orientation Parameters are required.

See [this reference](https://apps.dtic.mil/sti/pdfs/ADA637370.pdf) for a great discussion on the different coordinate frames.

## Test 1: Compare pure SGP4 propagation in the TEME frame.
1. Python + sgp4
2. Python + skyfield
3. C++ original function

We assume the 'wgs84' constants in all cases where applicable. We will use the following TLE:

In [1]:
line1 = "1 19548U 88091B   23110.52047465 -.00000310  00000+0  00000+0 0  9996"
line2 = "2 19548  13.4884 348.9767 0037807 339.0721  18.7948  1.00270407113813"

import numpy as np

# Parameters of propagation
year = 2023
mon = 4
day = 21
hr = 12
minute = 0
sec = 0

# Test 1.1, Python + sgp4
from sgp4.api import Satrec, jday
from sgp4.api import WGS72, WGS84

sat = Satrec.twoline2rv(line1, line2, WGS84)
jd, fr = jday(year, mon, day, hr, minute, sec)
e, r, v = sat.sgp4(jd, fr)
print(r, v)

# Test 1.2 Python + skyfield
from skyfield.api import EarthSatellite, load
from skyfield.sgp4lib import TEME

sfsat = EarthSatellite(line1, line2)
ts = load.timescale()
t = ts.utc(year, mon, day, hr, minute, sec)
pv = sfsat.at(t)
pos, vel = pv.frame_xyz_and_velocity(TEME)
print(pos.km, vel.km_per_s)

print("Error = %g" % (np.linalg.norm(np.array(r) - np.array(pos.km))))

(39641.16403399667, -13833.063014725132, -1428.375749762684) (1.0143747638499885, 2.8263194734421773, 0.7116770873234988)
[ 39641.17161091 -13833.0751021   -1428.37827126] [1.01437532 2.82632002 0.71167724]
Error = 0.014487


Clearly, there is a slight difference here, on the order of about 10m (0.01km). This is more than likely to be good enough, but why is it different if ```skyfield``` is only wrapping ```sgp4```? It turns out there's a simple explanation for this, which is pretty obvious looking at the code; ```skyfield``` defaults to using WGS72, and doesn't have an option to specify this in ```EarthSatellite()``` directly.

Note that it is _not at all obvious_ that the wrapper function ```Satrec.twoline2rv()``` takes in an optional parameter for the gravitational constant. It is apparent if you have looked at the original C++ code, which has an argument ```gravconsttype whichconst```, but otherwise since the python ```sgp4``` library wraps it and exposes an interface without good docstrings, the only documentation that shows this is at the PyPi site [here](https://pypi.org/project/sgp4/), scrolling down to the 'Gravity' section. Notably, it does claim that using WGS84 at this point in the propagator is likely to perform worse when compared to real measurements of satellite positions, due to the TLEs being generated with WGS72 models in mind.

In [2]:
# Test 1.1, Python + sgp4
from sgp4.api import Satrec, jday
from sgp4.api import WGS72, WGS84

sat = Satrec.twoline2rv(line1, line2, WGS72)
jd, fr = jday(year, mon, day, hr, minute, sec)
e, r, v = sat.sgp4(jd, fr)
print(r, v)

# Test 1.2 Python + skyfield
from skyfield.api import EarthSatellite, load
from skyfield.sgp4lib import TEME

sfsat = EarthSatellite(line1, line2)
ts = load.timescale()
t = ts.utc(year, mon, day, hr, minute, sec)
pv = sfsat.at(t)
pos, vel = pv.frame_xyz_and_velocity(TEME)
print(pos.km, vel.km_per_s)

print("Error = %g" % (np.linalg.norm(np.array(r) - np.array(pos.km))))

(39641.171610908845, -13833.075102099956, -1428.3782712556995) (1.0143753225229308, 2.826320024639999, 0.7116772422656449)
[ 39641.17161091 -13833.0751021   -1428.37827126] [1.01437532 2.82632002 0.71167724]
Error = 2.12967e-10


In [3]:
# Test 1.3 C++ original library
# Note that the original SGP4 twoline2rv function requires many more inputs, of which most are unnecessary.
# A good way to determine what to use is to look at what the python-sgp4 library did:
# SGP4Funcs::twoline2rv(line1, line2, ' ', ' ', 'i', whichconst,
#                          dummy, dummy, dummy, self->satrec);
# We see that the first two char options are unnecessary, and hence any dummy char can be placed. 'i' is to use the improved mode vs the older mode.
# The following 3 dummy variables are unused as we perform the propagation ourselves later on.
# Notably, the original code's propagator, sgp4(), only accepts time since epoch as the input.
# This means we have to find the difference between epoch time and the target time; this can be done by referencing the
# struct's 'jdsatepoch' & 'jdsatepochF' variables.
# Again, we can look at what the python-sgp4 library did:
# double tsince = (jd - satrec.jdsatepoch) * 1440.0
#                  + (fr - satrec.jdsatepochF) * 1440.0;
#
#
# Output (copied from my externally compiled and tested, you're gonna have to trust i did the above):
cpp_x = np.array([39641.171610909, -13833.075102099, -1428.378271256])
cpp_v = np.array([1.014375323, 2.826320025, 0.711677242])

print("Error = %g" % (np.linalg.norm(cpp_x - np.array(r))))


Error = 1.01447e-09


Can we in fact force ```skyfield.EarthSatellite``` to take in the WGS84 constants, like the ```sgp4``` library? Maybe?

In [4]:
class CustomSatellite(EarthSatellite):
    def __init__(self, line1, line2, name=None, ts=None, const=WGS72):
        super().__init__(line1, line2, name=name, ts=ts) # This ignores the const
        # So remake the satrec with the const now
        self.model = Satrec.twoline2rv(line1, line2, const)
        self._setup(self.model)

# Test 1.4, Python + sgp4
sat = Satrec.twoline2rv(line1, line2, WGS84)
jd, fr = jday(year, mon, day, hr, minute, sec)
e, r, v = sat.sgp4(jd, fr)
print(r, v)

# Test 1.5 Python + skyfield, repeated with custom class
sfsat = CustomSatellite(line1, line2, const=WGS84) # Set wgs84 here
ts = load.timescale()
t = ts.utc(year, mon, day, hr, minute, sec)
pv = sfsat.at(t)
pos, vel = pv.frame_xyz_and_velocity(TEME)
print(pos.km, vel.km_per_s)

print("Error = %g" % (np.linalg.norm(np.array(r) - np.array(pos.km))))

(39641.16403399667, -13833.063014725132, -1428.375749762684) (1.0143747638499885, 2.8263194734421773, 0.7116770873234988)
[ 39641.164034   -13833.06301473  -1428.37574976] [1.01437476 2.82631947 0.71167709]
Error = 2.11411e-10


Ah, so we can indeed restore the WGS84 functionality if we want! This is good news if we desire consistency in our frames later.

Alright, so far so good! Now we move on to the next task..

## Test 2: Converting TEME to ITRF

Note that the python ```sgp4``` library does not contain the coordinate transformation functions; this is only provided in ```skyfield```. This accurately reflects the original code, as the ```sgp4()``` function in C++ is contained in a separate header and source file (and even in a separate folder) from the coordinate transformations, found in the ```AstroLib``` folder.

Examining the C++ function ```AstroLib::teme_ecef``` shows that there are multiple extra inputs we require: 

```
double ttt, double jdut1, double lod, double xp, double yp
```

So we can split these into 2 separate sources:

1. ```double lod, double xp, double yp```
2. ```double ttt, double jdut1```

These are either simply extracted from the Earth Orientation Parameters (EOP) talked about previously, or derived from them. Here we shall use some archived values I have from the daily data (using Bulletin A) for 21/4/2023:

$$
xp = 0.007402 \text{arcsec}
$$
$$
yp = 0.452703 \text{arcsec}
$$
$$
lod = 0.7024 \text{msec}
$$
$$
dut1 = -0.0339525 \text{sec}
$$

We then can use one of the functions ```MathTimeLib::convtime()``` to extract the values in (2) by inputting the required ```dut1```, as well as a value ```dat```, defined as the difference between TAI and UTC; note that the docstring on the C++ function is somewhat misleading, putting it as ```utc - tai```, but the code itself shows that ```tai = utc + dat;```, hence ```dat``` should be strictly positive and is generally known as leap seconds. This doesn't change often. You can look at this [link](http://leapsecond.com/java/gpsclock.htm) to find out. For our test, we are using the value

$$
dat = 37 \text{sec}
$$

Okay, but what about the skyfield functions to convert into ITRF? It seems like there are no options to specify any of these, so how is it doing the conversions? It turns out, by looking at both the [documentation](https://rhodesmill.org/skyfield/accuracy-efficiency.html) and the source code itself, that you can in fact specify these using ```iers.install_polar_motion_table(ts, finals_data)```, and that if this is not done then the polar motion rotation matrix is simply not applied (or another way of looking at it is that if we set all the values to 0, then an identity matrix is generated and applied). 

The ```load.timescale()``` call also inherently stores defaults for ```dut1``` and ```delta_t``` (which is related to ```dat``` but compares TT to UT1 instead of UTC to TAI, see this [reference](https://rhodesmill.org/skyfield/api.html#time-scales)).

Reading the original source code for ```convtime()``` shows all the conversions:

1. ```ut1 = utc + dut1```
2. ```tai = utc + dat```
3. ```tt = tai + 32.184```

There's two tests we can perform here: one with the polar motion offsets and/or time information, and one without them.

1. Python + skyfield, no polar motion, and no time parameters i.e. ```dut1=0```
2. C++ original function, no polar motion (set all Earth Orientation Parameters to 0 including ```dut1```).
3. Python + skyfield, polar motion values set.
4. C++ original function, polar motion values set.

This is essentially examining two scenarios: the first two tests look at the case where there are no active updates from IERS EOP data (hence unable to set ```dut1``` and all polar motion parameters, and in effect ignoring all their related effects/set to 0), while the last two tests assume there are active updates and interpolation.



In [5]:
dat = 37

# Test 2.1, Python + skyfield, no polar motion
from skyfield.framelib import itrs

# We regenerate the timescale with specific data
ts = load.timescale(delta_t=dat+32.184) # here we set delta_t only using leap seconds, essentially fixing dut1 = 0
t = ts.utc(year, mon, day, hr, minute, sec)
print(t.delta_t)
print(t._leap_seconds())

# Then the propagation as before, note that we are using WGS84 constants now, from the previous cell
pv = sfsat.at(t)
pos, vel = pv.frame_xyz_and_velocity(TEME)

# And finally the conversion to ITRF
itrf_pos, itrf_vel = pv.frame_xyz_and_velocity(itrs)
print(itrf_pos.km, itrf_vel.km_per_s)

69.184
37.0
[ 27797.23166809 -31465.68654789  -1428.37574976] [-0.02657323 -0.05888965  0.71167709]


In [6]:
# Test 2.2 C++ original function teme_ecef, no polar motion, using wgs84
# Pseudo-code used:
# MathTimeLib::convtime(
#         year, month, day, hour, minute, second, 0, dut1, dat,
#         ut1, tut1, jdut1, jdut1Frac, utc, tai, tt, ttt, jdtt, jdttFrac,
#         tcg, tdb, ttdb, jdtdb, jdtdbFrac, tcb
#     ); // note that almost all the outputs are not used, except jdut1 & jdut1Frac
# AstroLib::teme_ecef(
#         r, v, a,
#         MathTimeLib::eTo,
#         recef, vecef, aecef,
#         ttt, // used as is from the output of convtime()
#         jdut1+jdut1Frac, // must remember to add in the fraction, common mistake to leave it out!
#         lod, xp, yp, // all set to 0 currently
#         0 // this last one is just an option to use extra correction terms, we pick no extra correction as it seems to agree with skyfield better
#     );

itrf_pos_cpp = np.array([27797.231668072, -31465.686547907, -1428.375749763])
itrf_vel_cpp = np.array([-0.026573275, -0.058889690, 0.711677087])

print("Error = %g" % (np.linalg.norm(itrf_pos_cpp - np.array(itrf_pos.km))))

Error = 2.43887e-08


This is great! The agreement is around 1e-8 km i.e. much less than 1m, which is better than we can strive for realistically anyway.

Some notes here:
1. Skyfield does a relatively good job of estimating ```delta_t``` internally (even from its old data), without you setting it.

In [7]:
ts_internal = load.timescale()
t_internal = ts_internal.utc(year, mon, day, hr, minute, sec)
print(t_internal.delta_t)
print(t.delta_t - t_internal.delta_t) # Difference between what we set and what it estimated

# Then the propagation as before
pv = sfsat.at(t_internal)
pos, vel = pv.frame_xyz_and_velocity(TEME)

# And finally the conversion to ITRF
itrf_pos, itrf_vel = pv.frame_xyz_and_velocity(itrs)
print(itrf_pos.km, itrf_vel.km_per_s)

print("Error = %g" % (np.linalg.norm(itrf_pos_cpp - np.array(itrf_pos.km))))

69.1786454
0.005354600000003984
[ 27797.21938188 -31465.6974017   -1428.37574976] [-0.02657325 -0.05888964  0.71167709]
Error = 0.0163938


So in this scenario, it's off by about 0.005s, which is not a lot, and that has increased the agreement error to about 16m. This is probably insignificant in most cases.

Conclusion here is that if the parameters are not provided, the results are likely to be significantly affected by just using skyfield's internal interpolation.

2. Skyfield uses the new iau2000a theories in the TEME to ECEF conversions. This can be traced by looking at the ```t.gast()``` call, which seems to lead eventually leads to the ```iau2000a``` call and many other relevant calls in ```nutationlib```. Overtly speaking, this does _not_ appear to be what is done in the C++ code for ```teme_ecef```, which strictly calls a ```gstime()``` function which has been documented as 'iau-82', and despite there being a ```gstime00()``` function which is documented as 'iau-2010', and has comments like ```//  greenwich mean sidereal time, iau 2000.```. It is likely that using this may bring the C++ code in greater agreement with the more current Skyfield code, but the fact that it does not cause any significant agreement error is somewhat puzzling.


In [8]:
dut1 = -0.0339525
xp = 0.007402 # arcsec
yp = 0.452703 # arcsec
lod = 0.7024 # msec
mjd = 60055.0 # the associated mjd at the extraction

# Test 2.3, Python + skyfield, polar motion and time setting
from skyfield.framelib import itrs

# We regenerate the timescale with specific data
ts = load.timescale(delta_t=dat+32.184-dut1) # here we set delta_t only using leap seconds, essentially fixing dut1 = 0

# Add polar motion
# iers.install_polar_motion_table() # We don't use this, but rather just set it ourselves.
# the source code suggests it internally makes a table and interpolates the coefficients at the later requested time
t = ts.utc(1858, 11, 17.0 + mjd)
ts.polar_motion_table = (
    np.array([t.tt, t.tt]),
    np.array([xp, xp]),
    np.array([yp, yp])
) # We trick the interpolate call by just putting the same data twice

t = ts.utc(year, mon, day, hr, minute, sec)

# Then the propagation as before
pv = sfsat.at(t)
pos, vel = pv.frame_xyz_and_velocity(TEME)

# And finally the conversion to ITRF
itrf_pos, itrf_vel = pv.frame_xyz_and_velocity(itrs)
print(itrf_pos.km, itrf_vel.km_per_s)

[ 27797.30952291 -31465.61458936  -1428.44580696] [-0.02657283 -0.05889127  0.71168133]


In [9]:
# Test 2.4 C++ original function teme_ecef, with polar motion
# Pseudo-code used:
# dut1 = -0.0339525;
# xp = 0.007402 / 3600.0 / 180.0 * pi; // convert to radians
# yp = 0.452703 / 3600.0 / 180.0 * pi;
# lod = 0.7024 * 1e-3; // convert to seconds
#
# MathTimeLib::convtime(
#         year, month, day, hour, minute, second, 0, dut1, dat,
#         ut1, tut1, jdut1, jdut1Frac, utc, tai, tt, ttt, jdtt, jdttFrac,
#         tcg, tdb, ttdb, jdtdb, jdtdbFrac, tcb
#     ); // note that almost all the outputs are not used, except jdut1 & jdut1Frac
# AstroLib::teme_ecef(
#         r, v, a,
#         MathTimeLib::eTo,
#         recef, vecef, aecef,
#         ttt, // used as is from the output of convtime()
#         jdut1+jdut1Frac, // must remember to add in the fraction, common mistake to leave it out!
#         lod, xp, yp, // all set to 0 currently
#         0 // this last one is just an option to use extra correction terms, we pick no extra correction as it seems to agree with skyfield better
#     );

itrf_pos_cpp_eop = np.array([27797.309531066, -31465.614582156, -1428.445806961])
itrf_vel_cpp_eop = np.array([-0.026573085, -0.058891301, 0.711676959])

print("Error = %g" % (np.linalg.norm(itrf_pos_cpp_eop - np.array(itrf_pos.km))))

Error = 1.08763e-05


Again, the agreement is less than 1m, which looks great!

Conclusion: despite the implementation differences in skyfield (likely using more updated theories like iau2000), these are not enough to warrant any significant change in results in the ITRF. Hence, for all intents and purposes, we can regard __skyfield implementation == C++ implementation__.

However, there is a clear difference between the results from adding in EOP parameters and completely ignoring them.


In [10]:
print("Error = %g" % (np.linalg.norm(itrf_pos_cpp_eop - itrf_pos_cpp)))

Error = 0.127082


This error of about 100m is much larger than the other agreement errors, and shows that the model may depend heavily on updated EOPs. However, this is still smaller than the usual TLE epoch position error...